In [ ]:
import os
os.environ['http_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128' 
os.environ['https_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128'
os.environ['PATH']=os.environ['PATH'] + ':/opt/ohpc/pub/apps/texlive/2023/bin/x86_64-linux/'

In [ ]:
import rebound
import numpy as np
import time
import random

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from matplotlib import rc
rc('text', usetex=True)
rc('font', size=10)
rc('legend', fontsize=8)
rc('text.latex', preamble=r'\usepackage{cmbright}')

In [ ]:
# Define arrays
p_values = np.linspace(25, 31, 20) # a has values between 2.1 to 2.5
ecc_values = np.linspace(0, 1, 20, endpoint=False)

a_values = p_values / (1 - ecc_values)

# Create pairs using nested loops and dictionary
pairs = [{'p': p, 'ecc': ecc, 'a': a} for ecc in ecc_values for p in p_values for a in a_values]

# Extract individual lists for plotting
p_list = [pair['p'] for pair in pairs]
ecc_list = [pair['ecc'] for pair in pairs]
a_list = [pair['a'] for pair in pairs]

print(min(p_list), max(p_list))
print(min(a_list), max(a_list))
# Plotting
plt.scatter(p_list, ecc_list, color='blue', alpha=0.5)
plt.xlabel('p values')
plt.ylabel('ecc values')
plt.title('Pairs of p and ecc values')
plt.grid(True)
plt.show()

In [ ]:
# Create rebound object
sim = rebound.Simulation()
 # Add Sun
sim.add('Sun')
# Add Jupiter 
sim.add('Neptune')               
# Add massless test particle
for i in range(400):
    sim.add(a=a_list[i], e=ecc_list[i], inc=random.uniform(0, np.pi)) # random inclination 
sim.move_to_com()

In [ ]:
sim.particles[400].orbit().a

In [ ]:
sim.status(showAllFields=False)

In [ ]:
%matplotlib inline
op = rebound.OrbitPlot(sim, unitlabel="[AU]", color=True, periastron=True)

In [ ]:
sim.dt = sim.particles[1].P*0.05
print(sim.dt) 

# Define integrators
integrators = ['mercurius', 'ias15', 'whfast']
sim.integrator = integrators[0]

In [ ]:
sim.save_to_file("/Net/Groups/BGI/scratch/ppandey/SDODynamics/data/Neptune_UniformDistData.bin", interval=1e3, delete_file=True)

In [ ]:
# Convert the orbital period in yrs
ref_period = (sim.particles[1].P / (2*np.pi))
print(f'Neptune period wrt Earth: {ref_period} yrs')

start = time.time()
# Integrating
sim.integrate(1e6)
end = time.time()
print(end - start)

In [ ]:
sim.status(showAllFields=False)

In [ ]:
data = rebound.Simulationarchive("/Net/Groups/BGI/scratch/ppandey/SDODynamics/experiments/uniform.bin")

eccentricity = []
time_steps = []

# Save the evolution of orbital elements
for i, sim in enumerate(data):
    eccentricity.append(sim.particles[2].orbit(primary=sim.particles[0]).e)
    time_steps.append(np.log10(data[i].t))

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(time_steps, eccentricity, s=5, c='red')
plt.title(r'Eccentricity Evolution')
plt.xlabel(r'$t$[yr]')
plt.ylabel(r'eccentricity')